In [1]:
import pandas as pd
import math
import os

In [32]:
def voto(x):
    try:
        "*" in x
        return False
    except:
        return True

def voto_troncato(x):
    return 0.5 * round(int(x * 100 / 25)/2 +0.1)
    
def correctDf(file):
    df = pd.DataFrame([])
#     df[["Cod.", "Ruolo", "Nome", "Voto", "Gf", "Gs", "Rp", "Rs", "Rf", "Au", "Amm", "Esp", "Ass", "Gdv", "Gdp"]] = pd.read_excel(file).dropna()
    df[["Cod.", "Ruolo", "Nome", "Voto", "Gf", "Gs", "Rp", "Rs", "Rf", "Au", "Amm", "Esp", "Ass"]] = pd.read_excel(file).dropna().iloc[: , :13]
    df = df.loc[df.Ruolo != 'ALL']
    df = df.loc[df.Voto.apply(voto)]
#     fv = [voto + 3*(Gf + Rp - Rs + Rf) - 2*Au - Gs - Esp + Ass - 0.5*Amm for _, _, _, voto, Gf, Gs, Rp, Rs, Rf, Au, Amm, Esp, Ass, _, _ in df.values]
    fv = [voto + 3*(Gf + Rp - Rs + Rf) - 2*Au - Gs - Esp + Ass - 0.5*Amm for _, _, _, voto, Gf, Gs, Rp, Rs, Rf, Au, Amm, Esp, Ass in df.values]
    df["FantaVoto"] = fv
    df["Nome"] = df.Nome.str.upper()
    return df

In [3]:
def voto1(lista):
    media = sum(lista)/len(lista)
    m, M = min(lista), max(lista)
    N = int(2*(M-m))
    lista_voti = [m + 0.5*i for i in range(N+1)]
    try:
        p = [math.factorial(N)/(math.factorial(int(2*(v-m)))*math.factorial(N-int(2*(v-m))))*(((2*(media-m)/N)**(2*(v-m))) * (1-(2*(media-m)/N))**(N-2*(v-m))) for v in lista_voti]
    except:
        p = [1]
    occorrenze = [p.count(x) for x in p]
    z = list(zip(p, lista_voti, occorrenze))
    z.sort(reverse=True)
    if z[0][2] > 1:
        return sum(list(map(lambda x: x[1], z[:z[0][2]])))/z[0][2]
    return z[0][1]

def voto2(lista, m=3, M=9):
    media = sum(lista)/len(lista)
    m, M = min(lista), max(lista)
    N = int(2*(M-m))
    lista_voti = [m + 0.5*i for i in range(N+1)]
    try:
        p = [math.factorial(N)/(math.factorial(int(2*(v-m)))*math.factorial(N-int(2*(v-m))))*(((2*(media-m)/N)**(2*(v-m))) * (1-(2*(media-m)/N))**(N-2*(v-m))) for v in lista_voti]
    except:
        p = [1]
    occorrenze = [p.count(x) for x in p]
    z = list(zip(p, lista_voti, occorrenze))
    z.sort(reverse=True)
    try:
        if z[1][2] > 1:
            if z[0][0] == z[1][0]:
                return sum(list(map(lambda x: x[1], z[:z[0][2]])))/z[0][2]
            return sum(list(map(lambda x: x[1], z[z[0][2]:z[0][2]+z[1][2]])))/z[1][2]
        return z[1][1]
    except:
        return None

def voto3(lista, m=3, M=9):
    media = sum(lista)/len(lista)
    m, M = min(lista), max(lista)
    N = int(2*(M-m))
    lista_voti = [m + 0.5*i for i in range(N+1)]
    try:
        p = [math.factorial(N)/(math.factorial(int(2*(v-m)))*math.factorial(N-int(2*(v-m))))*(((2*(media-m)/N)**(2*(v-m))) * (1-(2*(media-m)/N))**(N-2*(v-m))) for v in lista_voti]
    except:
        p = [1]
    occorrenze = [p.count(x) for x in p]
    z = list(zip(p, lista_voti, occorrenze))
    z.sort(reverse=True)
    try:
        if z[2][2] > 1:
            if z[0][0] == z[2][0]:
#                 print("sono uguale al primo")
                return sum(list(map(lambda x: x[1], z[:z[0][2]])))/z[0][2]
            if z[1][0] == z[2][0]:
#                 print("sono uguale al secondo")
                return sum(list(map(lambda x: x[1], z[z[0][2]:z[0][2]+z[1][2]])))/z[1][2]
#             print("sono diverso dal primo e dal secondo")
            return sum(list(map(lambda x: x[1], z[2:2+z[2][2]])))/z[2][2]
        return z[2][1]
    except:
        return None

def voto_potenziale(voto, fanta_voto, ruolo):
    if ruolo in ["D", "P"]:
        return (2*voto**2-21*voto+55)/4+fanta_voto
    return fanta_voto


def media(lista):
    return sum(lista)/len(lista)

In [4]:
def modello_fantacalcio(ultima_giornata, n_giornate, save=False, perc_presenze=0.375, file_quotazioni="Quotazioni_Fantacalcio_Stagione_2022_23.xlsx"):
    
#     excels = [f'Voti_Fantacalcio/Voti_Fantacalcio_Stagione_2020-21_Giornata_{i}.xlsx' for i in range(1,39)] + [f'Voti_Fantacalcio/Voti_Fantacalcio_Stagione_2021_22_Giornata_{i}.xlsx' for i in range(1,ultima_giornata+1)]
    path = f"{os.getcwd()}/Voti_Fantacalcio"
    excel_files = [file for (root, dirs, file) in os.walk(path)][0]
    excels = [f"Voti_Fantacalcio/{ex}" for ex in excel_files]
    list_df = [correctDf(ex) for ex in excels]
    
    stats = {}
    for df in list_df[-n_giornate:]:
        for ndarray in df.values:
            line = ndarray.tolist()
            nome = line[2]
            try:
                stats[nome]["GolFatti"].append(line[4] + line[8])
                stats[nome]["Assist"].append(line[12] + line[13])
                stats[nome]["GolSubiti"].append(line[5])
                stats[nome]["RigoriParati"].append(line[6])
                stats[nome]["RigoriSbagliati"].append(line[7])
                stats[nome]["Autogol"].append(line[9])
                stats[nome]["Gialli"].append(line[10])
                stats[nome]["Rossi"].append(line[11])
                stats[nome]["Voti"].append(line[3])
                stats[nome]["FantaVoti"].append(line[-1])
                stats[nome]["Partite"] += 1
            except:
                stats[nome] = {"R": line[1],
                               "GolFatti": [line[4] + line[8]],
                               "Assist": [line[12] + line[13]],
                               "GolSubiti": [line[5]],
                               "RigoriParati": [line[6]],
                               "RigoriSbagliati": [line[7]],
                               "Autogol": [line[9]],
                               "Gialli": [line[10]],
                               "Rossi": [line[11]],
                               "Voti": [line[3]],
                               "FantaVoti": [line[-1]],
                               "Partite": 1}
    a = pd.DataFrame.from_dict(stats).transpose()
    a["Media"] = a.Voti.apply(media)
    a["FantaMedia"] = a.FantaVoti.apply(media)
    
    a["Voto1"] = a.Voti.apply(voto1)
    a["Voto2"] = a.Voti.apply(voto2)
    a["Voto3"] = a.Voti.apply(voto3)
    a["FantaVoto1"] = a.FantaVoti.apply(voto1)
    a["FantaVoto2"] = a.FantaVoti.apply(voto2)
    a["FantaVoto3"] = a.FantaVoti.apply(voto3)
    a["VotoTroncato"] = a.Media.apply(voto_troncato)
    a["FantaVotoTroncato"] = a.FantaMedia.apply(voto_troncato)
    statistiche = a[["Partite", "Media", "FantaMedia", "Voto1", "Voto2", "Voto3", "FantaVoto1", "FantaVoto2", "FantaVoto3", "VotoTroncato", "FantaVotoTroncato"]]
    
    quotazioni = pd.read_excel(file_quotazioni, header=1)
    quotazioni["Nome"] = quotazioni["Nome"].str.upper()

    statistiche['Nome'] = statistiche.index
    pd.set_option('display.max_rows', None)
    result = pd.merge(quotazioni, statistiche, on="Nome", how= 'inner')
    result = result[["R", "Squadra"] + statistiche.columns.tolist()]
    result = result.loc[(result.Partite>=perc_presenze*n_giornate)]
    voti_potenziali = [voto_potenziale(v,f,r) for v,f,r in result[["Media", "FantaMedia", "R"]].values]
    voti_potenziali_t = [voto_potenziale(v,f,r) for v,f,r in result[["VotoTroncato", "FantaVotoTroncato", "R"]].values]
    result["VotoPotenziale"]=voti_potenziali
    result["VotoPotenzialeTroncato"]=voti_potenziali_t
    
    porta = result.loc[result.R == "P"]
    porta = porta.sort_values(["VotoPotenzialeTroncato", "VotoPotenziale"], ascending=(False, False))
    porta["Posizione"]=range(1, len(porta)+1)

    difesa = result.loc[result.R == "D"]
    difesa = difesa.sort_values(["VotoPotenzialeTroncato", "VotoPotenziale"], ascending=(False, False))
    difesa["Posizione"]=range(1, len(difesa)+1)

    centrocampo = result.loc[result.R == "C"]
    centrocampo = centrocampo.sort_values(["VotoPotenzialeTroncato", "VotoPotenziale"], ascending=(False, False))
    centrocampo["Posizione"]=range(1, len(centrocampo)+1)

    attacco = result.loc[result.R == "A"]
    attacco = attacco.sort_values(["VotoPotenzialeTroncato", "VotoPotenziale"], ascending=(False, False))
    attacco["Posizione"]=range(1, len(attacco)+1)
    
    header = ["R", "Nome", "Squadra", "Partite", "Media", "FantaMedia", "Voto1", "FantaVoto1", "Posizione", "VotoPotenziale", "VotoPotenzialeTroncato"]
    final_result = pd.concat([porta, difesa, centrocampo, attacco])[header]
    
    if save:
        
        path_final_result = f"final_result/giornata_{ultima_giornata}"
        file_final_result = f"final_result_ultime_{n_giornate}.xlsx"
        if not os.path.exists(path_final_result):
            os.makedirs(path_final_result)
            
        path_finale_final_result = os.path.join(path_final_result, file_final_result)
        
        # Create a Pandas Excel writer using XlsxWriter as the engine.
        writer = pd.ExcelWriter(path_finale_final_result, engine='xlsxwriter')

        # Write each dataframe to a different worksheet.
        final_result.to_excel(writer, index=False, sheet_name='final_result')
        porta[header].to_excel(writer, index=False, sheet_name='PORTIERI')
        difesa[header].to_excel(writer, index=False, sheet_name='DIFENSORI')
        centrocampo[header].to_excel(writer, index=False, sheet_name='CENTROCAMPISTI')
        attacco[header].to_excel(writer, index=False, sheet_name='ATTACCANTI')

        # Close the Pandas Excel writer and output the Excel file.
        writer.save()
        
        print(f"salvato final result {ultima_giornata} considerando le precedenti {n_giornate} in {path_finale_final_result}")

    return excels, final_result